In [5]:
txt_path = '../data/processed/processed_korquad.txt'
corpus = [sent.replace('\n', '').strip() for sent in open(txt_path, 'r', encoding='utf-8').readlines()]

In [6]:
corpus[0]

'1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

In [8]:
len(corpus)

76946

In [7]:
from sklearn.decomposition import TruncatedSVD
from soynlp.vectorizer import sent_to_word_contexts_matrix

In [9]:
input_matrx, idx2vocab = sent_to_word_contexts_matrix(corpus, windows=3, min_tf=10, dynamic_weight=True, verbose=True)

Create (word, contexts) matrix
  - counting word frequency from 76945 sents, mem=0.225 Gb
  - scanning (word, context) pairs from 76945 sents, mem=0.456 Gb
  - (word, context) matrix was constructed. shape = (25157, 25157)                    
  - done


* windows는 문맥의 길이
* min_tf는 최소 등장 횟수
* dynamic_weight는 중심 단어와 멀 수록 가중치를 감소하는지 여부
* verbose는 결과 출력 여부

In [10]:
cooc_svd = TruncatedSVD(n_components=100)
cooc_vecs = cooc_svd.fit_transform(input_matrx)

In [12]:
print('Before: {}, TruncatedSVD After: {}'.format(input_matrx.shape, cooc_vecs.shape))

Before: (25157, 25157), TruncatedSVD After: (25157, 100)


In [13]:
from soynlp.word import pmi

In [16]:
ppmi_matrix, _, _ = pmi(input_matrx, min_pmi=0)
ppmi_svd = TruncatedSVD(n_components=100)
ppmi_vecs = ppmi_svd.fit_transform(input_matrx)

In [17]:
print('Before: {}, PPMI TruncatedSVD After: {}'.format(input_matrx.shape, ppmi_vecs.shape))

Before: (25157, 25157), PPMI TruncatedSVD After: (25157, 100)


In [44]:
word2id = dict()
for word in idx2vocab:
    word2id[word]=len(word2id)

import torch
import numpy as np 

def similar_words(mat, word, k=10):
    eps =1e-6
    def cos_sim(a,b):
        return np.sum((a*b)/(np.linalg.norm(a)*np.linalg.norm(b)+eps),1)

    word_vec = mat[word2id[word]]
    
    cos_mat = cos_sim(word_vec, mat)
    sim, indices = torch.topk(torch.tensor(cos_mat),k+1)
    
    
    word_list = []
    for i in indices:
        if i != word2id[word]:
            word_list.append(idx2vocab[i])
    return word_list

In [48]:
similar_words(ppmi_vecs,'일본')

['수', '번째', '이', '두', '첫', '한', '그', '있다.', '것은', '있는', '몇']

In [49]:
similar_words(cooc_vecs,'일본')

['수', '번째', '이', '두', '첫', '한', '그', '있다.', '것은', '있는', '몇']